In [1]:
chori="dllec1239"# ETL Project 
# Due on 10/19/2019
# Project Team: Adelia Manuel, John Knight, Rashid Khokhar

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# CSV files downloaded from [Kaggle](https://www.kaggle.com/)
# Goal is to see if there is any significant price differenec in AirBnB pricing on 2 costs
in_file1 = "./Resources/AB_NYC_2019.csv"
in_file2="./Resources/seattle_01.csv"

### Extract Data Set 1 to DataFrames

In [4]:
in_df1 = pd.read_csv(in_file1)
in_df1.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


### Extract Data Set 2 to DataFrames

In [5]:
in_df2 = pd.read_csv(in_file2)
in_df2.head()

,Unnamed: 0,room_id,host_id,room_type,address,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,last_modified,latitude,longitude,location,name,currency,rate_type
0,0,2318,2536,Entire home/apt,"Seattle, WA, United States",21,5.0,8,4,2.5,250,46:14.7,47.610819,-122.290816,0101000020E6100000D449B6BA9C925EC0416326512FCE...,"Casa Madrona - Urban Oasis, 1 block from the P...",USD,nightly
1,1,3335,4193,Entire home/apt,"Seattle, WA, United States",1,NaN,4,2,1.0,100,08:45.3,47.529846,-122.275840,0101000020E61000006FBBD05CA7915EC04DF564FED1C3...,Sweet Seattle Urban Homestead 2 Bdr,USD,nightly
2,2,4291,35749,Private room,"Seattle, WA, United States",63,4.5,2,1,1.0,82,04:19.9,47.687801,-122.313427,0101000020E6100000BDAB1E300F945EC0FB93F8DC09D8...,Sunrise in Seattle Master Suite,USD,nightly
3,3,5682,8993,Entire home/apt,"Seattle, WA, United States",462,5.0,2,0,1.0,49,11:25.5,47.523980,-122.359891,0101000020E6100000FCC7427408975EC009E1D1C611C3...,"Cozy Studio, min. to downtown -WiFi",USD,nightly
4,4,6606,14942,Entire home/apt,"Seattle, WA, United States",134,4.5,2,1,1.0,90,12:38.4,47.654109,-122.337605,0101000020E6100000D47D00529B955EC07782FDD7B9D3...,"Fab, private seattle urban cottage!",USD,nightly


### Transformation of  Data Set 1 DataFrame

In [6]:
# Create a filtered dataframe from specific columns for Data Set #1 DataFrame
data1_cols = ["id", "room_type", "price", "number_of_reviews" ]
transformed_df1= in_df1[data1_cols].copy()
transformed_df1.head()

,id,room_type,price,number_of_reviews
0,2539,Private room,149,9
1,2595,Entire home/apt,225,45
2,3647,Private room,150,0
3,3831,Entire home/apt,89,270
4,5022,Entire home/apt,80,9


In [7]:
# Renaming Data Set #1 column headers to correspond with database table
transformed_df1 = transformed_df1.rename(columns={"id": "Facility_ID",
                                                  "number_of_reviews": "reviews"})

# Clean up of the data by dropping duplicates and setting the index
print("DataFrame Size Before Duplication Removal", transformed_df1.shape)
transformed_df1.drop_duplicates("Facility_ID", inplace=True)
print("DataFrame Size After Duplication Removal", transformed_df1.shape)
transformed_df1.set_index("Facility_ID", inplace=True)
transformed_df1.head()

DataFrame Size Before Duplication Removal (48895, 4)
DataFrame Size After Duplication Removal (48895, 4)


,room_type,price,reviews
Facility_ID,,,
2539,Private room,149,9
2595,Entire home/apt,225,45
3647,Private room,150,0
3831,Entire home/apt,89,270
5022,Entire home/apt,80,9


### Transformation of  Data Set 2 DataFrame

In [8]:
# Create a filtered dataframe from specific columns for Data Set #2 DataFrame
data2_cols = ["room_id", "room_type", "price","reviews" ]
transformed_df2= in_df2[data2_cols].copy()
transformed_df2.head()

,room_id,room_type,price,reviews
0,2318,Entire home/apt,250,21
1,3335,Entire home/apt,100,1
2,4291,Private room,82,63
3,5682,Entire home/apt,49,462
4,6606,Entire home/apt,90,134


In [9]:
# Renaming Data Set #2 column headers to correspond with database table
transformed_df2 = transformed_df2.rename(columns={"room_id": "Facility_ID"})

# Clean up of the data by dropping duplicates and setting the index
print("DataFrame Size Before Duplication Removal", transformed_df2.shape)
transformed_df2.drop_duplicates("Facility_ID", inplace=True)
print("DataFrame Size After Duplication Removal", transformed_df2.shape)
transformed_df2.set_index("Facility_ID", inplace=True)
transformed_df2.head()

DataFrame Size Before Duplication Removal (7576, 4)
DataFrame Size After Duplication Removal (7576, 4)


,room_type,price,reviews
Facility_ID,,,
2318,Entire home/apt,250,21
3335,Entire home/apt,100,1
4291,Private room,82,63
5682,Entire home/apt,49,462
6606,Entire home/apt,90,134


### Create database connection

In [10]:
connection_string = "postgres:bootcamp@localhost:5432/ETL_Proj_DB"
engine = create_engine(f'postgres+psycopg2://{connection_string}')

In [11]:
# Confirm tables
engine.table_names()

# Note: 
# The tables ny_100 and se_100 were created using only 100 records from each table 
# This was done for quicker checks of SQL statements and to verify solutions

['NY', 'Seatle', 'ny_100', 'se_100']

### Load DataFrames into database

In [12]:
transformed_df1.to_sql(name='NY', con=engine, if_exists='append', index=True)

In [13]:
len(transformed_df1)

48895

In [14]:
transformed_df1['room_type'].value_counts()


Entire home/apt    25409
Private room       22326
Shared room         1160
Name: room_type, dtype: int64

In [15]:
list(transformed_df1['room_type'].unique())

['Private room', 'Entire home/apt', 'Shared room']

In [16]:
transformed_df2.to_sql(name='Seatle', con=engine, if_exists='append', index=True)

In [17]:
len(transformed_df2)

7576

In [18]:
transformed_df2['room_type'].value_counts()

Entire home/apt    5603
Private room       1844
Shared room         129
Name: room_type, dtype: int64

In [19]:
list(transformed_df2['room_type'].unique())

['Entire home/apt', 'Private room', 'Shared room']

##### >>>>>>>>>>>>>>  Rest of the manipulation is to be done in PostgressSQL <<<<<<<<<<<<<<<<<<